# 為甚麼需要格式化的答案
1. 自動擷取資訊: 從語言模型的輸出的自然語言，自動解析 (parse) 出我們想要的結構化資料。
   
2. 讓語言模型可以呼叫 function/API: 讓語言模型自動產出該 function/api 的參數 (arguments)

In [1]:
from langchain.prompts import ChatPromptTemplate
from langchain.schema import SystemMessage, AIMessage, HumanMessage
from langchain_setup import ChatOpenAI

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "你是一個道地的台灣人，你來到了手搖飲料店，想要買一杯手搖飲。"),
        ("human", "你好，請問今天要喝甚麼?"),
        ("ai", "請給我一杯{choice}。"),
        ("human", "中杯大杯?甜度冰塊?"),
    ]
)

model = ChatOpenAI()

response1 = model(prompt.format_messages(choice="台茶17號白鷺紅茶"))
print("Response1: ", response1.content)
response2 = model(prompt.format_messages(choice="桂花烏龍鮮奶茶"))
print("Response2: ", response2.content)

Response1:  中杯，甜度七分，冰塊正常。
Response2:  中杯，甜度正常，少冰。


但假設今天我們想要自動整理點單資訊的話，我們會想要得到這樣的形式

In [2]:
[
    {
        "甜度": 5,
        "冰塊": "少",
        "大小": "中",
    },
    {
        "甜度": 5,
        "冰塊": "正常",
        "大小": "中",
    },
]

[{'甜度': 5, '冰塊': '少', '大小': '中'}, {'甜度': 5, '冰塊': '正常', '大小': '中'}]

# 2. Format Instructions (Langchain 功能)

Langchain 實作的 Outpur parser 提供了兩種東西：

1. *Format Instruction*: 指導 LLM 依照某個形式回答的文字，讓使用者可以插入 prompt 裡面
   
2. *Parser*: LLM 產出該形式的文字後，解析器 (parser) 依照形式讀取成想要的資料格式

In [1]:
from typing import Literal
from langchain.prompts import ChatPromptTemplate
from langchain.schema import SystemMessage, AIMessage, HumanMessage
from langchain_setup import ChatOpenAI
from langchain.output_parsers import StructuredOutputParser, ResponseSchema, PydanticOutputParser
from langchain.pydantic_v1 import BaseModel, Field

# 定義 Output parser

## 透過 Pydantic
class Drink(BaseModel):
    sweetness: int = Field(description="飲料的甜度。介於 1 到 10。")
    ice: Literal['去冰', '少冰', '正常'] = Field(description="冰塊的量")
    size: Literal['中杯','大杯'] = Field(description="飲料杯的大小")
pydantic_parser = PydanticOutputParser(pydantic_object=Drink)

## 或透過 Langchain 自訂的 Schema
response_schemas = [
    ResponseSchema(name="甜度", description="飲料的甜度。介於 1 到 10。", type='integer'),
    ResponseSchema(name="冰塊", description="冰塊的量。可以是「去冰」、「少冰」、「正常」的其中一個。", type='string'),
    ResponseSchema(name="大小", description="飲料杯的大小。可以是「中杯」、「大杯」的其中一個。", type='string'),
]
structure_parser = StructuredOutputParser.from_response_schemas(response_schemas)

# 編輯 Prompt
prompt_template = ChatPromptTemplate.from_messages(
    [
        ("system", "你是一個道地的台灣人，你來到了手搖飲料店，想要買一杯手搖飲。"),
        ("human", "你好，請問今天要喝甚麼?"),
        ("ai", "請給我一杯{choice}。"),
        ("human", "中杯大杯?甜度冰塊? 請依照以下指示回答\n{format_instructions}\n"),
    ],
)
## 插入形式指引 (format instructions), 這邊的 `partial` 類似 functools.partial
prompt_template2 = prompt_template.partial(format_instructions=structure_parser.get_format_instructions())

# 實際執行看看
messages = prompt_template2.format_messages(choice="黑糖珍珠奶茶")
model = ChatOpenAI(temperature=0) # 通常需要遵照格式時，會取消隨機性以避免跳脫格式
response_message = model(messages)
print(structure_parser.parse(response_message.content))

{'甜度': 5, '冰塊': '正常', '大小': '中杯'}


首先來看看什麼是「形式指引 (format instruction)」

In [2]:
print(structure_parser.get_format_instructions())

The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"甜度": integer  // 飲料的甜度。介於 1 到 10。
	"冰塊": string  // 冰塊的量。可以是「去冰」、「少冰」、「正常」的其中一個。
	"大小": string  // 飲料杯的大小。可以是「中杯」、「大杯」的其中一個。
}
```


In [3]:
print(pydantic_parser.get_format_instructions())

The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"properties": {"sweetness": {"title": "Sweetness", "description": "\u98f2\u6599\u7684\u751c\u5ea6\u3002\u4ecb\u65bc 1 \u5230 10\u3002", "type": "integer"}, "ice": {"title": "Ice", "description": "\u51b0\u584a\u7684\u91cf", "enum": ["\u53bb\u51b0", "\u5c11\u51b0", "\u6b63\u5e38"], "type": "string"}, "size": {"title": "Size", "description": "\u98f2\u6599\u676f\u7684\u5927\u5c0f", "enum": ["\u4e2d\u676f", "\u5927\u676f"], "type": "string"}}, "required": ["sweetness", "ice", "size"]}
```


把形式指引 (format instruction) 插入原本的 Prompt 裡

In [4]:
print(prompt_template2.format(choice='黑糖珍珠奶茶'))

System: 你是一個道地的台灣人，你來到了手搖飲料店，想要買一杯手搖飲。
Human: 你好，請問今天要喝甚麼?
AI: 請給我一杯黑糖珍珠奶茶。
Human: 中杯大杯?甜度冰塊? 請依照以下指示回答
The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"甜度": integer  // 飲料的甜度。介於 1 到 10。
	"冰塊": string  // 冰塊的量。可以是「去冰」、「少冰」、「正常」的其中一個。
	"大小": string  // 飲料杯的大小。可以是「中杯」、「大杯」的其中一個。
}
```



以這種 Prompt Engineering 的方式，讓模型產出符合格式的文字。

In [5]:
print(response_message.content)

```json
{
	"甜度": 5,
	"冰塊": "正常",
	"大小": "中杯"
}
```


最後將結果從文字 (str) 形式解析 (parse) 成目標資料形式 (data structure)

In [6]:
parsed = structure_parser.parse(response_message.content)
print(parsed)
print(type(parsed))

{'甜度': 5, '冰塊': '正常', '大小': '中杯'}
<class 'dict'>


其他還有各種現成的 output parser

而雖然我們叫模型遵守格式，但它畢竟是一個不可控的黑箱子，還是可能不遵守格式。此時有些 output parser 就有實作一些邏輯來處理這樣的狀況。

In [7]:
from langchain.output_parsers import (
    ListOutputParser,
    DatetimeOutputParser,
    EnumOutputParser,
    PydanticOutputParser,  # 可以設定值的資料形式，和檢查值的邏輯
    OutputFixingParser,  # 如果輸出格式錯誤，就把錯誤格式的輸出＋格式指令丟到一個ＬＬＭ去改
    RetryOutputParser,  # 如果輸出格式錯誤，就把原本的prompt+錯誤格式的輸出＋格式指令丟到一個ＬＬＭ再試一次
)

# 3. Function Calling (provided by OpenAI)
- 支援 API version 2023-07-01-preview 後，gpt-4-0613 和 gpt-3.5-turbo-0613 模型

- 模型有特別為甚麼時候該呼叫甚麼函式或不呼叫函式微調過

- 底層是將函式以模型學習過的格式插入 system prompt 裡，所以是會占用 token 數的 [(OpenAI 官方說明)](https://platform.openai.com/docs/guides/gpt/function-calling)

- 當然還是有機會輸出不守規矩

- 基本上 Langchain output parser 和 OpenAI function calling 都可以做到同樣的事，只是 OpenAI 的模型有特別為此微調過

## 3.1 OpenAI 定義的參數

主要有兩大參數
- `functions`: 定義可以用的函式 (functions)

- `function_call`: 挑選函式 (function) 的模式

  - `none`: 自然回答，絕不為函式 (function) 產生呼叫 (calling) 用參數

  - `auto`: 自動決定要不要為函式產生呼叫用的參數、要針對哪個函式。
  
  - `{"name": <function name>}`: 必定要針對這個函式，產生呼叫用的參數。

In [1]:
from pprint import pprint
from langchain.schema import HumanMessage
from langchain_setup import ChatOpenAI
from copy import deepcopy

openai_kwargs = {
    "functions": [
        # 第一個函式
        {
            # 函式名稱
            "name": "take_leave",  # 請假
            # 函式敘述
            "description": "連接到公司系統的請假 API",
            # 函式參數
            "parameters": {
                "type": "object",
                "properties": {
                    "start_date": {
                        "title": "Start date",
                        "description": "請假開始日期",
                        "type": "string",
                    },
                    "n_days": {
                        "title": "Number of days",
                        "description": "請假天數",
                        "type": "integer",
                    },
                    "reason": {
                        "title": "Reason",
                        "description": "想要請假的理由",
                        "type": "string",
                    },
                },
                # 必需參數
                "required": ["start_date", "end_date"],
            },
        },
        {
            # 函式名稱
            "name": "away_on_official_business",  # 填公出
            # 函式敘述
            "description": "因公司業務需要離開辦公室超過二十分鐘時即須填寫公出單，此函式可以協助您填寫並送出公出單。",
            # 函式參數
            "parameters": {
                "type": "object",
                "properties": {
                    "date": {
                        "title": "Date",
                        "description": "公出日期",
                        "type": "string",
                    },
                    "destination": {
                        "title": "Destination",
                        "description": "公出目的地",
                        "enum": ["外部", "松山大樓", "中崙大樓"],  # 設定合法值域
                    },
                    "reason": {
                        "title": "Reason",
                        "description": "公出的理由",
                        "type": "string",
                    },
                },
                # 必需參數
                "required": ["date", "destination"],
            },
        },
    ],
    # 
    "function_call": "auto",
}

model = ChatOpenAI(temperature=0)

自動決定要針對哪個函式 (function) 產生呼用的參數 (arguments)。

若模型決定為某個函式產生參數，會回傳
```
{
    'function_call':{
        'arguments': <參數和參數的值，以 str 形式的 dict 表示>, 
        'name': <想要呼叫的函式名>
    }
}
```

In [2]:
query = "幫我從2023年父親節開始請兩天假，並用一個容易審查通過的理由。"
response = model(messages=[HumanMessage(content=query)], **openai_kwargs)
display(response)
print(response.additional_kwargs['function_call']['name'])
print(response.additional_kwargs['function_call']['arguments'])

AIMessage(content='', additional_kwargs={'function_call': {'name': 'take_leave', 'arguments': '{\n  "start_date": "2023-06-18",\n  "n_days": 2,\n  "reason": "我想在父親節這個特別的日子，和我的父親一起度過寶貴的時間，表達我的愛和感激之情。這是一個難得的機會，我希望能夠好好珍惜。"\n}'}})

take_leave
{
  "start_date": "2023-06-18",
  "n_days": 2,
  "reason": "我想在父親節這個特別的日子，和我的父親一起度過寶貴的時間，表達我的愛和感激之情。這是一個難得的機會，我希望能夠好好珍惜。"
}


同時也會決定是否想要呼叫函式 (function)

In [3]:
query = "想要加入洛琪希教要去哪裡報名?"
response = model(messages=[HumanMessage(content=query)], **openai_kwargs)
display(response)

AIMessage(content='您可以透過洛琪希教的官方網站或是臉書粉絲專頁進行報名。在官方網站或臉書粉絲專頁上，您可以找到相關的報名資訊和報名方式。請確認洛琪希教的官方網站或臉書粉絲專頁以獲取最新的報名資訊。')

我們可以強迫其一定要針對某一個函式產生呼叫用的參數

In [4]:
kwargs = deepcopy(openai_kwargs)
kwargs["function_call"] = {"name": "away_on_official_business"} # 強迫填公出
query = "幫我從2023年父親節開始請兩天假，並用一個容易審查通過的理由。"
response = model(messages=[HumanMessage(content=query)], **kwargs)
print(response.additional_kwargs['function_call']['name'])
print(response.additional_kwargs['function_call']['arguments'])

away_on_official_business
{
  "date": "2023-06-18",
  "destination": "家中",
  "reason": "陪伴家人慶祝父親節"
}


強迫其必須自然回答，不要嘗試呼叫任何函式

In [5]:
kwargs = deepcopy(openai_kwargs)
kwargs["function_call"] = "none"
query = "幫我從2023年父親節開始請兩天假，並用一個容易審查通過的理由。"
response = model(
    messages=[HumanMessage(content=query)], **kwargs
)
print(response.content)

好的，我會幫您請兩天假，並提供一個容易審查通過的理由。請稍等片刻。


## 3.2 自動產生 OpenAI functions 參數

每次都要寫記得怎麼寫又臭又長的參數太累了，讓 `convert_to_openai_function` 幫我們代勞

In [1]:
import warnings
from pprint import pprint
from typing import Literal
from langchain.prompts import ChatPromptTemplate
from langchain.chains.openai_functions.base import convert_to_openai_function
from langchain.output_parsers.openai_functions import PydanticOutputFunctionsParser
from langchain.pydantic_v1 import BaseModel, Field, root_validator
from langchain_setup import ChatOpenAI

用一般的函式 (function) 轉換成 openai funciton 參數。注意目前 typing 只能是 `int`, `float`, `str`, `bool`, `pydantic.BaseModel` 的其中一種。該參數 (argument) 如果是不支援的 typing 就會直接變成沒有 type 要求。(例如下面的 `size` 所使用的 `Literal`)

In [2]:
def order_drink(sweetness: int, ice: str, size: Literal['中杯', '大杯']) -> str:
    """在台灣點手搖飲料

    Args:
        sweetness: 飲料甜度。
        ice: 冰塊多寡。可以是 '去冰','少冰','正常冰' 的其中一種。
        size: 飲料大小。
    """

pprint(convert_to_openai_function(order_drink))

{'description': '在台灣點手搖飲料',
 'name': 'order_drink',
 'parameters': {'properties': {'ice': {'description': '冰塊多寡。可以是 '
                                                      "'去冰','少冰','正常冰' 的其中一種。",
                                       'type': 'string'},
                               'size': {'description': '飲料大小。'},
                               'sweetness': {'description': '飲料甜度。',
                                             'type': 'number'}},
                'required': ['sweetness', 'ice', 'size'],
                'type': 'object'}}


利用 pydantic 模型 (類似 dataclass) 的表述轉換成 openai funciton 參數。目前 Langchain 比較推用這個方式表述，支援也比較好。(例子中可以看到把 `Literal` 轉換成 `enum`)

In [3]:
class Drink(BaseModel):
    """在台灣點手搖飲料"""

    sweetness: int = Field(description='飲料甜度。')
    ice: Literal['去冰','少冰','正常冰'] = Field(description='冰塊多寡。')
    size: Literal['中杯', '大杯'] = Field(description='飲料大小')

pprint(convert_to_openai_function(Drink))

{'description': '在台灣點手搖飲料',
 'name': 'Drink',
 'parameters': {'description': '在台灣點手搖飲料',
                'properties': {'ice': {'description': '冰塊多寡。',
                                       'enum': ['去冰', '少冰', '正常冰'],
                                       'title': 'Ice',
                                       'type': 'string'},
                               'size': {'description': '飲料大小',
                                        'enum': ['中杯', '大杯'],
                                        'title': 'Size',
                                        'type': 'string'},
                               'sweetness': {'description': '飲料甜度。',
                                             'title': 'Sweetness',
                                             'type': 'integer'}},
                'required': ['sweetness', 'ice', 'size'],
                'title': 'Drink',
                'type': 'object'}}


配合專用的輸出解析 (output parsing) ㄧ起服用體驗更好。使用 `PydanticOutputFunctionsParser` 將 OpenAI 回傳的 OpenAI function 回答 parse 成對應的 pydantic 格式 (schema)。

當然如果 pydantic 模型有設值 (value) 檢查 (validation) 或後處理 (postprocessing) 的話就會在解析 (parsing)並建立 pydantic 的資料模型 (BaseModel) 時觸發

下面的例子是通常實際應用時會做的事

In [4]:
# ======================================================
# 定義 schema, 檢查 (validation), 後處理 (postprocessing)
# ======================================================
class Drink(BaseModel):
    """在台灣點手搖飲料"""

    sweetness: int = Field(description='飲料甜度。')
    ice: Literal['去冰','少冰','正常冰'] = Field(description='冰塊多寡。')
    size: Literal['中杯', '大杯'] = Field(description='飲料大小')
    
    # 更多其他功能可以查詢 pydantic 的文檔 (documentation)
    @root_validator() # 類似 dataclass 的 __postprocess__ 
    def validate_values(cls, values: dict) -> dict:
        if not (1 <= values['sweetness'] <= 10):
            warnings.warn(f"甜度 {values['sweetness']} 不在 [1,10] 內，自動調整...")
            values['sweetness'] = max(min(1, values['sweetness']), 10)
        print("嗶嗶！檢查/後處理完畢！")
        return values

# 對應的 ouput parser
output_parser = PydanticOutputFunctionsParser(pydantic_schema=Drink) # 傳入剛才定好的 pydantic schema

# ======================================================
# 定義 input template
# ======================================================
prompt_template = ChatPromptTemplate.from_messages(
    [
        ("system", "你是一個道地的台灣人，你來到了手搖飲料店，想要買一杯手搖飲。"),
        ("human", "你好，請問今天要喝甚麼?"),
        ("ai", "請給我一杯{choice}。"),
        ("human", "中杯大杯?甜度冰塊? 請依照指示的格式回答"), # 指示的格式會由 OpenAI API 透過 System message 插入
    ],
)

# ======================================================
# 模型
# ======================================================
model = ChatOpenAI(temperature=0)

# ======================================================
# Chain 起來
# ======================================================
function_kwargs = convert_to_openai_function(Drink)
openaifn_kwargs = {'functions': [function_kwargs], 'function_call': {'name': 'Drink'}}
# 將 input prompt、model、output parsing 自動串起來的語法，之後會詳細介紹
chain = prompt_template | model.bind(**openaifn_kwargs) | output_parser

# ======================================================
# 可以拿來用了
# ======================================================
output = chain.invoke({"choice": "海神奶茶"})
print("輸出的資料形式是 pydantic schema:", type(output))
print({'甜度': output.sweetness, '冰塊': output.ice, '大小': output.size})

嗶嗶！檢查/後處理完畢！
輸出的資料形式是 pydantic schema: <class '__main__.Drink'>
{'甜度': 10, '冰塊': '正常冰', '大小': '中杯'}


C:\Users\121664\AppData\Local\Temp\ipykernel_30872\1319550851.py:15: UserWarning: 甜度 50 不在 [1,10] 內，自動調整...
  warnings.warn(f"甜度 {values['sweetness']} 不在 [1,10] 內，自動調整...")
